In [43]:
import os
import openai
import sys

env = {}

# read .env file and set variables
with open('.env') as f:
    for line in f:
        key, value = line.strip().split('=')
        env[key] = value

openai.api_key = env['OPEN_AI_KEY']
OPENAI_API_KEY = env['OPEN_AI_KEY']

In [2]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("docs/Lecture1.pdf"),
    PyPDFLoader("docs/Lecture2.pdf"),
    PyPDFLoader("docs/Lecture3.pdf"),
    PyPDFLoader("docs/Lecture4.pdf"),
    PyPDFLoader("docs/Lecture5.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 284 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 264 0 (offset 0)


In [3]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [4]:
splits = text_splitter.split_documents(docs)

In [5]:
len(splits)

154

In [44]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=env['OPEN_AI_KEY'])

In [18]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [20]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [21]:
import numpy as np

In [22]:
np.dot(embedding1, embedding2)

0.9630350414845892

In [23]:
np.dot(embedding1, embedding3)

0.7701147991091337

In [24]:
np.dot(embedding2, embedding3)

0.7591130000177129

In [25]:
pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 10.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 9.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 11.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 11.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [26]:
from langchain.vectorstores import Chroma

In [27]:
persist_directory = 'docs/chroma/'

In [28]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [29]:
print(vectordb._collection.count())

154


In [30]:
question = "is there an email i can ask for help"

In [31]:
docs = vectordb.similarity_search(question,k=3)

In [32]:
len(docs)

3

In [33]:
docs[0].page_content

'Andrew NgSPAM'

In [34]:
vectordb.persist()

/Users/pikkuaapo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [35]:
question = "what did they say about matlab?"

In [36]:
docs = vectordb.similarity_search(question,k=5)

In [37]:
docs[0]

Document(metadata={'page': 2, 'source': 'docs/Lecture5.pdf'}, page_content='Octave Tutorial \nComputing on data \nMachine Learning')

In [38]:
docs[1]

Document(metadata={'page': 5, 'source': 'docs/Lecture5.pdf'}, page_content='Octave Tutorial \nVectorial implementation \nMachine Learning')

In [39]:
question = "what did they say about regression in the third lecture?"

In [40]:
docs = vectordb.similarity_search(question,k=5)

In [41]:
for doc in docs:
    print(doc.metadata)

{'page': 12, 'source': 'docs/Lecture4.pdf'}
{'page': 8, 'source': 'docs/Lecture4.pdf'}
{'page': 5, 'source': 'docs/Lecture4.pdf'}
{'page': 17, 'source': 'docs/Lecture4.pdf'}
{'page': 21, 'source': 'docs/Lecture4.pdf'}


In [42]:
print(docs[4].page_content)

  variables		
  equa2on	
  Learning
